In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
from joblib import load

In [2]:
class CrustaceanIdentifierSystem:
    def __init__(self, img_size=(224, 224), num_classes=5):
        self.img_size = img_size
        self.class_names = ['blue_swimming_crab','mud_crab','tiger_prawn','whiteleg_shrimp','river_crab']

    def extract_color_features(self, image):
        """Extract color-based features from image"""
        if len(image.shape) == 3 and image.shape[2] == 3:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        else:
            image_rgb = image
        resized = cv2.resize(image_rgb, (64, 64))
        hist_r = cv2.calcHist([resized], [0], None, [32], [0, 256])
        hist_g = cv2.calcHist([resized], [1], None, [32], [0, 256])
        hist_b = cv2.calcHist([resized], [2], None, [32], [0, 256])

        # Calculate mean, standard, std, and skewness of color channels.
        mean_colors = np.mean(resized.reshape(-1, 3), axis=0)
        std_colors = np.std(resized.reshape(-1, 3), axis=0)
        skew_colors = np.array([
            np.mean(((resized[:, :, i] - mean_colors[i]) / std_colors[i])**3)
            for i in range(3)
        ])
        features = np.concatenate([
            hist_r.flatten(), hist_g.flatten(), hist_b.flatten(),
            mean_colors, std_colors, skew_colors
        ])
        
        return features

    def extract_shape_features(self, image):
        """Extract shape-based features from image"""
        # Convert to grayscale for shape analysis
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        # Resize for consistent feature extraction
        gray_resized = cv2.resize(gray, (128, 128))
        
        # Apply Gaussian blur to reduce noise
        blurred = cv2.GaussianBlur(gray_resized, (5, 5), 0)
        
        # Threshold to create binary image
        _, binary = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        shape_features = []
        
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Basic shape measurements
            area = cv2.contourArea(largest_contour)
            perimeter = cv2.arcLength(largest_contour, True)
            
            # Aspect ratio and extent
            x, y, w, h = cv2.boundingRect(largest_contour)
            aspect_ratio = float(w) / h if h != 0 else 0
            rect_area = w * h
            extent = float(area) / rect_area if rect_area != 0 else 0
            
            # Solidity (convex hull ratio)
            hull = cv2.convexHull(largest_contour)
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area != 0 else 0
            
            # Circularity
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter != 0 else 0
            
            # Moments for shape description
            moments = cv2.moments(largest_contour)
            if moments['m00'] != 0:
                # Centroid
                cx = int(moments['m10'] / moments['m00'])
                cy = int(moments['m01'] / moments['m00'])
                # Normalized central moments (Hu moments)
                hu_moments = cv2.HuMoments(moments).flatten()
                # Take log to make them more manageable
                hu_moments = -np.sign(hu_moments) * np.log10(np.abs(hu_moments) + 1e-10)
            else:
                hu_moments = np.zeros(7)
            
            # Equivalent diameter
            equiv_diameter = np.sqrt(4 * area / np.pi) if area > 0 else 0
            
            # Compactness
            compactness = (perimeter ** 2) / (4 * np.pi * area) if area > 0 else 0
            
            # Roundness
            roundness = (4 * np.pi * area) / (perimeter ** 2) if perimeter > 0 else 0
            
            # Eccentricity (using fitted ellipse)
            if len(largest_contour) >= 5:
                ellipse = cv2.fitEllipse(largest_contour)
                major_axis = max(ellipse[1])
                minor_axis = min(ellipse[1])
                eccentricity = np.sqrt(1 - (minor_axis / major_axis) ** 2) if major_axis > 0 else 0
            else:
                eccentricity = 0
            
            # Combine all shape features
            shape_features = [
                area / 10000,  # Normalized area
                perimeter / 1000,  # Normalized perimeter
                aspect_ratio,
                extent,
                solidity,
                circularity,
                equiv_diameter / 100,  # Normalized equivalent diameter
                compactness,
                roundness,
                eccentricity,
                *hu_moments  # 7 Hu moments
            ]
            
        else:
            # If no contours found, return zero features
            shape_features = [0] * 17  # 10 basic features + 7 Hu moments
        
        return np.array(shape_features)

    def extract_texture_features(self, image):
        """Extract texture-based features using Local Binary Pattern (LBP)"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
        
        gray_resized = cv2.resize(gray, (64, 64))
        
        # Simple LBP implementation
        def local_binary_pattern(img, radius=1, neighbors=8):
            h, w = img.shape
            lbp = np.zeros((h, w), dtype=np.uint8)
            
            for i in range(radius, h - radius):
                for j in range(radius, w - radius):
                    center = img[i, j]
                    pattern = 0
                    for k in range(neighbors):
                        angle = 2 * np.pi * k / neighbors
                        x = int(i + radius * np.cos(angle))
                        y = int(j + radius * np.sin(angle))
                        if 0 <= x < h and 0 <= y < w:
                            if img[x, y] >= center:
                                pattern |= (1 << k)
                    lbp[i, j] = pattern
            return lbp
        
        lbp = local_binary_pattern(gray_resized)
        
        # Calculate histogram of LBP
        hist, _ = np.histogram(lbp.ravel(), bins=32, range=(0, 256))
        hist = hist.astype(float)
        hist /= (hist.sum() + 1e-10)  # Normalize
        
        # Additional texture features
        # Contrast, homogeneity, energy
        glcm_features = []
        
        # Simple contrast measure
        contrast = np.var(gray_resized)
        
        # Homogeneity (inverse difference moment)
        homogeneity = np.mean(gray_resized) / (np.std(gray_resized) + 1e-10)
        
        # Energy (uniformity)
        energy = np.sum(hist ** 2)
        
        texture_features = np.concatenate([
            hist,  # LBP histogram (32 features)
            [contrast / 1000, homogeneity / 100, energy]  # Additional features (3 features)
        ])
        
        return texture_features

    def extract_combined_features(self, image):
        """Extract combined color, shape, and texture features"""
        color_features = self.extract_color_features(image)
        shape_features = self.extract_shape_features(image)
        texture_features = self.extract_texture_features(image)
        
        # Combine all features
        combined_features = np.concatenate([
            color_features,
            shape_features,
            texture_features
        ])
        
        return combined_features

In [3]:
# Crustacean Identifier System class
crustacean_system = CrustaceanIdentifierSystem()

# Load trained model
model = load_model(r"C:\Users\pc\_djohn_files\_model.keras")

# Load saved scaler and kmeans
scaler = load(r"C:\Users\pc\_djohn_files\_scaler.pkl")
kmeans_model = load(r"C:\Users\pc\_djohn_files\_kmeans.pkl")

C:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 50 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [4]:
# Print inputs
print([inp.name for inp in model.inputs])

['image_input', 'cluster_input']


In [5]:
# Path to unseen dataset
test_dir = r"D:\_stress-test-case\andoid-test-images"

In [8]:
import time

# Collect ground truth and predictions
y_true = []
y_pred = []


# Define test resolutions
resolutions = [(640, 480), (1280, 720), (1920, 1080)]

# Collect results
results = []

for class_name in crustacean_system.class_names:
    class_dir = os.path.join(test_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"Warning: {class_dir} not found, skipping...")
        continue

    for img_file in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_file)

        # Load original image
        img = cv2.imread(img_path)
        if img is None:
            print(f"Could not read {img_path}, skipping...")
            continue

        for res in resolutions:
            # Resize to test resolution
            img_resized_test = cv2.resize(img, res)

            # Start timing
            start_time = time.time()

            # Preprocess for CNN input
            img_resized = cv2.resize(img_resized_test, crustacean_system.img_size)
            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)
            img_normalized = img_rgb / 255.0
            img_batch = np.expand_dims(img_normalized, axis=0)

            # Extract cluster input
            combined_features = crustacean_system.extract_combined_features(img_resized_test)
            features_scaled = scaler.transform([combined_features])
            cluster_label = kmeans_model.predict(features_scaled)[0]
            cluster_input = tf.keras.utils.to_categorical(
                [cluster_label], num_classes=kmeans_model.n_clusters
            )

            # Predict
            prediction = model.predict(
                {"image_input": img_batch, "cluster_input": cluster_input}, verbose=0
            )
            predicted_class = np.argmax(prediction)

            # End timing
            end_time = time.time()
            prediction_time_ms = (end_time - start_time) * 1000

            results.append({
                "Image": img_file,
                "Species": class_name,
                "Resolution": f"{res[0]}x{res[1]}",
                "Predicted": crustacean_system.class_names[predicted_class],
                "Time (ms)": round(prediction_time_ms, 2)
            })

In [9]:
# Print results
import pandas as pd
df = pd.DataFrame(results)
print(df)

          Image             Species Resolution           Predicted  Time (ms)
0   bsc (1).jpg  blue_swimming_crab    640x480  blue_swimming_crab     566.89
1   bsc (1).jpg  blue_swimming_crab   1280x720  blue_swimming_crab     540.60
2   bsc (1).jpg  blue_swimming_crab  1920x1080  blue_swimming_crab     668.80
3   bsc (2).jpg  blue_swimming_crab    640x480  blue_swimming_crab     537.00
4   bsc (2).jpg  blue_swimming_crab   1280x720  blue_swimming_crab     493.99
5   bsc (2).jpg  blue_swimming_crab  1920x1080  blue_swimming_crab     495.00
6   bsc (3).jpg  blue_swimming_crab    640x480  blue_swimming_crab     484.99
7   bsc (3).jpg  blue_swimming_crab   1280x720  blue_swimming_crab     449.99
8   bsc (3).jpg  blue_swimming_crab  1920x1080  blue_swimming_crab     457.99
9   bsc (4).jpg  blue_swimming_crab    640x480  blue_swimming_crab     453.00
10  bsc (4).jpg  blue_swimming_crab   1280x720  blue_swimming_crab     450.00
11  bsc (4).jpg  blue_swimming_crab  1920x1080  blue_swimming_cr

In [10]:
# --- Evaluation ---
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=crustacean_system.class_names))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))


Classification Report:
                    precision    recall  f1-score   support

blue_swimming_crab       1.00      1.00      1.00         4
          mud_crab       1.00      1.00      1.00         4
       tiger_prawn       1.00      1.00      1.00         4
   whiteleg_shrimp       1.00      1.00      1.00         4
        river_crab       1.00      1.00      1.00         4

          accuracy                           1.00        20
         macro avg       1.00      1.00      1.00        20
      weighted avg       1.00      1.00      1.00        20


Confusion Matrix:
[[4 0 0 0 0]
 [0 4 0 0 0]
 [0 0 4 0 0]
 [0 0 0 4 0]
 [0 0 0 0 4]]
